<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/maehara250question_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **前原の250問　前処理**

## **スマホ画像resize**

ここはローカルでやる（ファイルサイズが大きいため）

### PNGでリサイズ (今回はjpgでやるので省略)

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/スマホ"

# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/スマホ_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
png_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(".png")]

with tqdm(total=len(png_files), unit="file") as pbar:
    for filename in png_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # リサイズした画像を新しいディレクトリに保存
            new_image_path = os.path.join(new_directory, filename)
            resized_img.save(new_image_path)

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 240/240 [02:06<00:00,  1.90file/s]

リサイズが完了しました。


### JPGでリサイズ

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/スマホ"
# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/スマホ_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
jpg_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(".png")]

with tqdm(total=len(jpg_files), unit="file") as pbar:
    for filename in jpg_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # 画像をRGBモードに変換
            rgb_img = resized_img.convert("RGB")

            # リサイズした画像を新しいディレクトリに保存
            new_filename = os.path.splitext(filename)[0] + ".jpg"  # ファイル名をjpg形式に変更
            new_image_path = os.path.join(new_directory, new_filename)
            rgb_img.save(new_image_path, "JPEG")  # JPEG形式で保存

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 240/240 [01:16<00:00,  3.14file/s]

リサイズが完了しました。


### 対応表にないものを削除

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# 対応表のCSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/対応表2022.xlsx - Sheet1.csv"
df = pd.read_csv(csv_file)

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/スマホ_resized"

# ディレクトリ内の画像ファイルを取得
image_files = [filename for filename in os.listdir(image_dir) if filename.lower().endswith((".jpg", ".png"))]

# tqdmを使用してファイル処理の進捗を表示
for filename in tqdm(image_files, desc="Processing files"):
    # ファイル名から拡張子を除いたベース名を取得
    basename, extension = os.path.splitext(filename)

    # 対応表からimg_idがベース名と一致する行を検索
    matched_rows = df[df["img_id"] == basename]

    # 一致する行が見つかった場合
    if not matched_rows.empty:
        # 最初の一致行のbasenameを取得
        new_basename = str(matched_rows.iloc[0]["basename"])  # 文字列に変換

        # 新しいファイル名を生成（拡張子を維持）
        new_filename = new_basename + extension

        # ファイルをリネーム
        old_path = os.path.join(image_dir, filename)
        new_path = os.path.join(image_dir, new_filename)
        os.rename(old_path, new_path)

        # Matchしたファイルのメッセージを表示
        tqdm.write(f"Matched and renamed: {filename} -> {new_filename}")
    else:
        # 対応表にないファイルを削除
        file_path = os.path.join(image_dir, filename)
        os.remove(file_path)

        # 削除したファイルのメッセージを表示
        tqdm.write(f"No match found. Deleted: {filename}")

### さらにリストにないものを削除

In [ ]:
import os
import pandas as pd

# CSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/updated_corr_table_modified3.csv"
df = pd.read_csv(csv_file, encoding='shift_jis')

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/スマホ_resized"

# ディレクトリ内のファイルを取得
files = os.listdir(image_dir)

# CSVファイルをbasename列で辞書化
csv_dict = df.set_index('basename').to_dict(orient='index')

# 一致するベース名を検索し、一致しないファイルを削除
matching_basenames = []
for file in files:
    basename = os.path.splitext(file)[0]
    if basename in csv_dict:
        matching_basenames.append(basename)
    else:
        file_path = os.path.join(image_dir, file)
        os.remove(file_path)
        print(f"一致するベース名が見つからないため、ファイルを削除しました: {file}")

# 結果を出力
if matching_basenames:
    print("一致するベース名が見つかったファイル:")
    for basename in matching_basenames:
        print(basename)
else:
    print("一致するベース名が見つかりませんでした。")

print(f"一致するベース名が見つかったファイルの数： {len(matching_basenames)}")

## **フォトスリット画像Resize**

フォトスリットもダウンロードしてリサイズする

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# 元のディレクトリのパスを指定
original_directory = "/Users/kitaguchi/Downloads/フォトスリット"

# 新しいディレクトリのパスを指定
new_directory = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# 新しいディレクトリが存在しない場合は作成
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# 元のディレクトリ内のすべてのファイルを処理
image_extensions = (".jpg", ".png", ".PNG")
image_files = [filename for filename in os.listdir(original_directory) if filename.lower().endswith(image_extensions)]

with tqdm(total=len(image_files), unit="file") as pbar:
    for filename in image_files:
        # 画像を開く
        image_path = os.path.join(original_directory, filename)
        with Image.open(image_path) as img:
            # 画像のサイズを取得
            width, height = img.size

            # 横幅を640pxに設定し、縦幅を比率に合わせて計算
            new_width = 640
            new_height = int(height * (new_width / width))

            # 画像をリサイズ
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)

            # 画像をRGBモードに変換
            rgb_img = resized_img.convert("RGB")

            # リサイズした画像を新しいディレクトリに保存
            new_filename = os.path.splitext(filename)[0] + ".jpg"  # ファイル名をjpg形式に変更
            new_image_path = os.path.join(new_directory, new_filename)
            rgb_img.save(new_image_path, "JPEG")  # JPEG形式で保存

        pbar.update(1)

print("リサイズが完了しました。")

100%|██████████| 238/238 [00:16<00:00, 14.61file/s]

リサイズが完了しました。


In [ ]:
import os
import pandas as pd

# CSVファイルを読み込む
csv_file = "/Users/kitaguchi/Downloads/updated_corr_table_modified3.csv"
df = pd.read_csv(csv_file, encoding='shift_jis')

# 画像ファイルがあるディレクトリのパス
image_dir = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# ディレクトリ内のファイルを取得
files = os.listdir(image_dir)

# CSVファイルをbasename列で辞書化
csv_dict = df.set_index('basename').to_dict(orient='index')

# 一致するベース名を検索し、一致しないファイルを削除
matching_basenames = []
for file in files:
    basename = os.path.splitext(file)[0]
    if basename in csv_dict:
        matching_basenames.append(basename)
    else:
        file_path = os.path.join(image_dir, file)
        os.remove(file_path)
        print(f"一致するベース名が見つからないため、ファイルを削除しました: {file}")

# 結果を出力
if matching_basenames:
    print("一致するベース名が見つかったファイル:")
    for basename in matching_basenames:
        print(basename)
else:
    print("一致するベース名が見つかりませんでした。")

print(f"一致するベース名が見つかったファイルの数： {len(matching_basenames)}")

### スマホ&スリットで欠損している画像を同定

In [ ]:
import os

dir1 = "/Users/kitaguchi/Downloads/スマホ_resized"
dir2 = "/Users/kitaguchi/Downloads/フォトスリット_resized"

# dir1とdir2のファイル一覧を取得
files1 = os.listdir(dir1)
files2 = os.listdir(dir2)

# dir1のファイルをループ
for file1 in files1:
    # 拡張子を除いたファイル名を取得
    basename1 = os.path.splitext(file1)[0]

    # dir2のファイルをループ
    found = False
    for file2 in files2:
        # 拡張子を除いたファイル名を取得
        basename2 = os.path.splitext(file2)[0]

        # basenameが一致すればfound=Trueにしてループを抜ける
        if basename1 == basename2:
            found = True
            break

    # dir2で一致するファイルが見つからなかった場合、パスを表示
    if not found:
        print(os.path.join(dir1, file1))

# dir2のファイルをループ
for file2 in files2:
    # 拡張子を除いたファイル名を取得
    basename2 = os.path.splitext(file2)[0]

    # dir1のファイルをループ
    found = False
    for file1 in files1:
        # 拡張子を除いたファイル名を取得
        basename1 = os.path.splitext(file1)[0]

        # basenameが一致すればfound=Trueにしてループを抜ける
        if basename2 == basename1:
            found = True
            break

    # dir1で一致するファイルが見つからなかった場合、パスを表示
    if not found:
        print(os.path.join(dir2, file2))

--> 不一致ファイルを削除し、updated_corr_table_modified4.csvとして保存

In [ ]:
#画像に黒塗り白字の番号をマージ（Claude3が読み取りやすいようにするため）

from PIL import Image, ImageDraw, ImageFont
import os

# 入力ディレクトリと出力ディレクトリのパスを指定
input_dir = "/Users/kitaguchi/Desktop/前原の240問/スリット_serial"
output_dir = "/Users/kitaguchi/Desktop/前原の240問/スリット_serial_merge"

# 出力ディレクトリが存在しない場合は作成
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ファイル名を描画するためのフォントを指定
font = ImageFont.load_default(36)

# 入力ディレクトリ内のすべてのJPGファイルを処理
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        # 画像を開く
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # 画像に描画するためのDrawオブジェクトを作成
        draw = ImageDraw.Draw(img)

        # ファイル名を描画する座標を指定
        text_position = (10, 10)

        # 黒塗りのボックスを描画
        bbox = draw.textbbox(text_position, filename, font=font)
        draw.rectangle(bbox, fill=(0, 0, 0))

        # ファイル名を黒塗りのボックス上に描画
        draw.text(text_position, filename, font=font, fill=(255, 255, 255))

        # 新しいファイル名を生成して保存
        new_filename = os.path.join(output_dir, filename)
        img.save(new_filename)

print("処理が完了しました。")

## ここでスマホ画像をgdridveにuploadする --> ここからはgdrive上でやる

## **前原の100問を追加**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
maehara100_dir = "/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/前原Dr_100問"

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

maehara100_dir = "/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/前原Dr_100問"
maehara100dir = "/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/maehara100_resized"

# maehara100dirが存在しない場合は作成
if not os.path.exists(maehara100dir):
    os.makedirs(maehara100dir)

# maehara100_dirの画像をリサイズしてmaehara100dirに保存
image_files = [filename for filename in os.listdir(maehara100_dir) if filename.lower().endswith(('.jpg', '.png', '.jpeg'))]
total_images = len(image_files)

with tqdm(total=total_images, unit='image') as pbar:
    for filename in image_files:
        filepath = os.path.join(maehara100_dir, filename)
        img = Image.open(filepath)

        # 画像を640pxにリサイズ（アスペクト比を維持）
        img.thumbnail((640, 640))

        # RGBAモードの場合はRGBモードに変換
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        # リサイズした画像をJPEG形式で保存
        new_filename = os.path.splitext(filename)[0] + '.jpg'
        new_filepath = os.path.join(maehara100dir, new_filename)
        img.save(new_filepath, 'JPEG')

        pbar.update(1)

print("画像のリサイズが完了しました。")

100%|██████████| 100/100 [00:43<00:00,  2.29image/s]

画像のリサイズが完了しました。


#**前原の100問をリネームして前原250問に統合**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# CSVファイルのパスを指定する
csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/updated_corr_table_modified4.csv'

# CSVファイルを読み込んでデータフレームを作成する
df = pd.read_csv(csv_path, encoding='shift_jis')

# データフレームの最初の5行を表示する
print(df.head())

In [ ]:
import os

# フォルダのパスを指定する
folder_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_resized'

# フォルダ内のファイル名を取得する（拡張子を除く）
files_in_folder = [os.path.splitext(file)[0] for file in os.listdir(folder_path)]

# データフレームの 'basename' 列の値を取得する
basenames_in_df = df['basename'].tolist()

# 一致するファイル名を保存するリストを初期化する
matching_files = []

# データフレームの 'basename' 列の値がフォルダ内のファイル名と一致するかどうかを調べる
for basename in basenames_in_df:
    if basename in files_in_folder:
        matching_files.append(basename)

# 一致するファイル名の数を表示する
print(f"一致するファイル名の数: {len(matching_files)}")

一致するファイル名の数: 184


In [ ]:
# CSVファイルのパスを指定する
csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv'

# CSVファイルを読み込んでデータフレームを作成する（エンコーディングを指定）
df_answer = pd.read_csv(csv_path, encoding='utf-8')

# データフレームの最初の5行を表示する
print(df_answer.head())

In [ ]:
classification_mapping = {
    'infection': 0,
    'normal': 1,
    'non-infection': 2,
    'scar': 3,
    'tumor': 4,
    'deposit': 5,
    'APAC': 6,
    'lens-opacity': 7,
    'bullous': 8
}

df_answer['classification_num'] = df_answer['classification'].map(classification_mapping)

In [ ]:
import os
import pandas as pd

# ディレクトリのパスを指定
directory = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara100/maehara100_slit'

# ディレクトリ内のファイル名を取得
file_names = os.listdir(directory)

# 番号を格納するリストを初期化
numbers = []

# ファイル名から番号を抽出してリストに追加
for file_name in file_names:
    if file_name.endswith('.jpg'):
        number = int(file_name.split('_')[1].split('.')[0])
        numbers.append(number)

# 番号リストを小さい順に並べ替え
sorted_numbers = sorted(numbers)

# df_answerの中で、numberがsorted_numbersリストと一致する行を抜き出す
df_answer_slit = df_answer[df_answer['number'].isin(sorted_numbers)]

# 抜き出した行を表示
print(df_answer_slit)

In [ ]:
import os
import pandas as pd

# ディレクトリのパスを指定
directory = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara100/maehara100_slit'

# ディレクトリ内のファイル名を取得
file_names = os.listdir(directory)

# 番号を格納するリストを初期化
numbers = []

# ファイル名から番号を抽出してリストに追加
for file_name in file_names:
   if file_name.endswith('.jpg'):
       number = int(file_name.split('_')[1].split('.')[0])
       numbers.append(number)

# 番号リストを小さい順に並べ替え
sorted_numbers = sorted(numbers)

# df_answerの中で、numberがsorted_numbersリストと一致する行を抜き出す
df_answer_slit = df_answer[df_answer['number'].isin(sorted_numbers)]

# 新しいデータフレームを作成
new_data = pd.DataFrame({
   'basename': df_answer_slit['number'].astype(str),
   'class_num': df_answer_slit['classification_num'],
   'disease_English': df_answer_slit['classification'],
   'top1': '',
   'top1_prob': '',
   'top2': '',
   'top2_prob': '',
   'top3': '',
   'top3_prob': '',
})

# 元のデータフレームと新しいデータを結合
df = pd.concat([df, new_data], ignore_index=True)

# 通し番号の列を追加
df.insert(0, 'serial_number', range(1, len(df) + 1))

# 結果を表示
print(df)

In [ ]:
df_sumaho_total = df.copy()

import re

def replace_basename_sumaho(basename):
    if pd.isna(basename):
        return basename
    basename = str(basename)
    if re.match(r'^\d+$', basename):
        return f'sumaho_{int(basename):03d}'
    else:
        return basename

df_sumaho_total['basename'] = df_sumaho_total['basename'].apply(replace_basename_sumaho)

In [ ]:
df_slit_total = df.copy()

def replace_basename_slit(basename):
    if pd.isna(basename):
        return basename
    basename = str(basename)
    if re.match(r'^\d+$', basename):
        return f'slit_{int(basename):03d}'
    else:
        return basename

df_slit_total['basename'] = df_slit_total['basename'].apply(replace_basename_slit)

In [ ]:
# CSVファイルとして保存
df_sumaho_total.to_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara_sumaho_total.csv', index=False)
df_slit_total.to_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara_slit_total.csv', index=False)

In [ ]:
import pandas as pd

# CSVファイルのパスを指定（適宜変更してください）
csv_file_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara_slit_total.csv'

# CSVファイルを読み込んでデータフレームを作成
df = pd.read_csv(csv_file_path)

# データフレームの先頭5行を表示
print(df.head())

In [ ]:
import os
import shutil
import pandas as pd

# CSVファイルのパスを指定（適宜変更してください）
csv_file_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara_slit_total.csv'

# CSVファイルを読み込んでデータフレームを作成
df = pd.read_csv(csv_file_path)

# データフレームの先頭5行を表示
print(df.head())

# 新しいフォルダのパスを指定
new_folder_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial'

# フォルダが存在する場合は削除する
if os.path.exists(new_folder_path):
    shutil.rmtree(new_folder_path)
    print(f"既存のフォルダ '{new_folder_path}' が削除されました。")

# 新しいフォルダを作成する
os.makedirs(new_folder_path)
print(f"新しいフォルダ '{new_folder_path}' が作成されました。")

# 画像フォルダのパスを指定
image_folders = [
    '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_resized',
    '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara100/maehara100_slit'
]

# 画像がコピーされたかどうかを追跡するフラグ
image_copied = False

# 画像フォルダ内のファイルを処理
for folder in image_folders:
    for filename in os.listdir(folder):
        basename, extension = os.path.splitext(filename)

        # basenameがデータフレームに存在するか確認
        if basename in df['basename'].values:
            # 一致する行を取得
            row = df[df['basename'] == basename].iloc[0]

            # 新しいファイル名を作成
            new_filename = f"{row['serial_number']}{extension}"

            # 画像をコピーしてリネーム
            src_path = os.path.join(folder, filename)
            dst_path = os.path.join(new_folder_path, new_filename)
            shutil.copy(src_path, dst_path)

            print(f"画像 '{filename}' を '{new_filename}' としてコピーしました。")
            image_copied = True

        else:
            print(f"画像 '{filename}' が見つかりませんでした")

In [ ]:
import os
import shutil

import pandas as pd

# CSVファイルのパスを指定（適宜変更してください）
csv_file_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara_sumaho_total.csv'

# CSVファイルを読み込んでデータフレームを作成
df = pd.read_csv(csv_file_path)

# データフレームの先頭5行を表示
print(df.head())

# 新しいフォルダのパスを指定
new_folder_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial'

# フォルダが存在する場合は削除する
if os.path.exists(new_folder_path):
    shutil.rmtree(new_folder_path)
    print(f"既存のフォルダ '{new_folder_path}' が削除されました。")

# 新しいフォルダを作成する
os.makedirs(new_folder_path)
print(f"新しいフォルダ '{new_folder_path}' が作成されました。")

# 画像フォルダのパスを指定
image_folders = [
    '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_resized',
    '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/maehara100/maehara100_sumaho'
]

# 画像がコピーされたかどうかを追跡するフラグ
image_copied = False

# 画像フォルダ内のファイルを処理
for folder in image_folders:
    for filename in os.listdir(folder):
        basename, extension = os.path.splitext(filename)

        # basenameがデータフレームに存在するか確認
        if basename in df['basename'].values:
            # 一致する行を取得
            row = df[df['basename'] == basename].iloc[0]

            # 新しいファイル名を作成
            new_filename = f"{row['serial_number']}{extension}"

            # 画像をコピーしてリネーム
            src_path = os.path.join(folder, filename)
            dst_path = os.path.join(new_folder_path, new_filename)
            shutil.copy(src_path, dst_path)

            print(f"画像 '{filename}' を '{new_filename}' としてコピーしました。")
            image_copied = True

        else:
            print(f"画像 '{filename}' が見つかりませんでした")
